# 00 라이브러리

In [1]:
from konlpy.tag import Okt
from konlpy.utils import pprint

# FlashText
from flashtext.keyword import KeywordProcessor

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np

import re
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-1-ac016749b071>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 01 경로설정

In [2]:
year = 2019
path = "./data/NaverNews{}/".format(year)
load_path = path + "/NaverNewsCleaning{}.csv".format(year)
save_path = path + "/NaverNewsPOS{}.csv".format(year)

# 02 DataLoad

In [3]:
df = pd.read_csv(load_path)

In [4]:
df = df.dropna()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4376 entries, 0 to 4375
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     4376 non-null   object
 1   Company  4376 non-null   object
 2   Title    4376 non-null   object
 3   Similar  4376 non-null   object
dtypes: object(4)
memory usage: 170.9+ KB


In [6]:
df.head(5)

,Date,Company,Title,Similar
0,2019.10.01.,머니투데이,"""앗 따가워!"" 정전기 잘 통하는 사람들의 공통점",김대리가 생활 속 꿀팁을 전합니다 엄마 아빠 싱글족 직장인 등 다양한 모습의 김대리...
1,2019.10.01.,뉴시스,태풍 '미탁' 영향으로 목포항구축제 하루 늦춰 4일 개막,지난해 목포항구축제 행사장에 설치된 어등터널 photonewsis 제 태풍 미탁의 ...
2,2019.10.08.,머니투데이,또 돌아온 환절기… 영양제 꼭 먹어야 하나,김대리가 생활 속 꿀팁을 전합니다 엄마 아빠 싱글족 직장인 등 다양한 모습의 김대리...
3,2019.10.08.,스포츠동아,"아시아나, 대만 가오슝 정기노선 취항",부터 전환클룩과 이벤트도 진행아시아나항공이 인천가오슝 구간을 정기 노선으로 운항한다...
4,2019.10.08.,매일경제,페이스북의 위대한 혹은 무모한 도전 `리브라`,페이스북 리브라사진디스트리트 서울에 사는 김 모군은 이번 가을에 뉴욕으로 여행을 가...


# 03 유사어 처리

In [7]:
 words_dic = {
         "강경화 외교부 장관" : "외교부장관",
         "강경화 장관" : "외교부장관",
         "이일병 연세대 명예교수" : "이일병",
         "강경화 외교부 장관의 남편" : "이일병",
         "외교부장관 배우자" : "이일병",
         "강경화 배우자" : "이일병",
         "강경화 남편" : "이일병",
         "이 교수" : "이일병",
         "조국 교수" : "조국",
         "조국 법무부 장관" : "조국",
         
         "미국 대선" : "미국대선",
         
         # 호칭관련 단어
         "아빠" : "아버지",
         "엄마" : "어머니",
     
         # 일반 단어
         "BTS" : "방탄소년단",
         "배낭여행족" : "배낭여행자",
         "배낭여행객" : "배낭여행자",
         "배낭여행자" : "배낭여행자",
         "관광 비행" : "관광비행",
         "저비용 항공사" : "저비용항공사",
         "도착지 없는 비행" : "관광비행",
         "상상여행" : "관광비행",
         "세계여행" : "해외여행",
         "불매 운동" : "불매운동",
         "비무장 지대" : "비무장지대",
         "투자 회사" : "투자회사",
         "베트남 전쟁" : "베트남전쟁",
         "종합 평가" : "종합평가",
         "세계 문화유산" : "세계문화유산",
         "건강 보험" : "건강보험",
        
         # 사람이름
         "보엠 푸치니" : "보엠푸치니",
         "보엠" : "보엠푸치니",
     
         # 나라이름
         "타이베이" : "타이베이",
         "타이페이" : "타이베이",
     
         # 시,도명칭
         "강원도" : "강원",
         "경상북도" : "경북",
         "경상남도" : "경남",
         "충청북도" : "충북",
         "충청남도" : "충남",
         "전라북도" : "전북",
         "전라남도" : "전남",
         "제주도" : "제주",
          
         # 정부기관명
         "기획재정부" : "기재부",
         "보건복지부" : "복지부",
         "소방재난본부" : "소방본부",
         "국토교통부" : "국토부",
         "인천공항" : "인천국제공항",
         "문화체육관광부" : "문체부",
         "한국관광공사" : "관광공사",
        
         # 의료 관련 단어
         "예방 접종" : "예방접종",
         "요양 병원" : "요양병원",
     
         # 코로나 관련 단어
         "코로나" : "코로나19",
         "코로나 바리러스" : "코로나19",
         "사회적 거리두기" : "사회적거리두기",
         "확진 판정" : "확진판정",
         
         # 항공사 이름
         "사우스웨스트 항공" : "사우스웨스트항공",
         "제트스타 항공" : "제트스타항공",
         "에어아시아 엑스" : "에어아시아엑스",
     
         # 창업관련
         "밴처 기업" : "밴처기업",
         "예비 창업" : "예비창업",
        
    }

In [8]:
def chang_words(data, wordDic):
    Keyword_processor = KeywordProcessor()
    for key in wordDic:
        Keyword_processor.add_keyword(key, wordDic[key])
    word_data = Keyword_processor.replace_keywords(data)
    return word_data  

In [9]:
df["Similar_Body"] = df.progress_apply(lambda x : chang_words(x["Similar"], words_dic), axis=1)

# 04 Bag of Word

In [10]:
def word_pos(data):
    okt = Okt()
    return list(okt.nouns(data))

In [11]:
df["noun"] = df["Similar_Body"].progress_apply(word_pos)

In [12]:
df.head(20)

,Date,Company,Title,Similar,Similar_Body,noun
0,2019.10.01.,머니투데이,"""앗 따가워!"" 정전기 잘 통하는 사람들의 공통점",김대리가 생활 속 꿀팁을 전합니다 엄마 아빠 싱글족 직장인 등 다양한 모습의 김대리...,김대리가 생활 속 꿀팁을 전합니다 어머니 아버지 싱글족 직장인 등 다양한 모습의 김...,"[김대, 리가, 생활, 속, 꿀팁, 전, 어머니, 아버지, 싱글, 족, 직장인, 등..."
1,2019.10.01.,뉴시스,태풍 '미탁' 영향으로 목포항구축제 하루 늦춰 4일 개막,지난해 목포항구축제 행사장에 설치된 어등터널 photonewsis 제 태풍 미탁의 ...,지난해 목포항구축제 행사장에 설치된 어등터널 photonewsis 제 태풍 미탁의 ...,"[지난해, 목포항, 축제, 행사, 설치, 어등, 터널, 제, 태풍, 탁, 영향, 목..."
2,2019.10.08.,머니투데이,또 돌아온 환절기… 영양제 꼭 먹어야 하나,김대리가 생활 속 꿀팁을 전합니다 엄마 아빠 싱글족 직장인 등 다양한 모습의 김대리...,김대리가 생활 속 꿀팁을 전합니다 어머니 아버지 싱글족 직장인 등 다양한 모습의 김...,"[김대, 리가, 생활, 속, 꿀팁, 전, 어머니, 아버지, 싱글, 족, 직장인, 등..."
3,2019.10.08.,스포츠동아,"아시아나, 대만 가오슝 정기노선 취항",부터 전환클룩과 이벤트도 진행아시아나항공이 인천가오슝 구간을 정기 노선으로 운항한다...,부터 전환클룩과 이벤트도 진행아시아나항공이 인천가오슝 구간을 정기 노선으로 운항한다...,"[부터, 전환클룩, 이벤트, 진행, 아시아나, 항공, 인천, 가오슝, 구간, 정기,..."
4,2019.10.08.,매일경제,페이스북의 위대한 혹은 무모한 도전 `리브라`,페이스북 리브라사진디스트리트 서울에 사는 김 모군은 이번 가을에 뉴욕으로 여행을 가...,페이스북 리브라사진디스트리트 서울에 사는 김 모군은 이번 가을에 뉴욕으로 여행을 가...,"[페이스북, 리브, 스트리트, 서울, 김, 군, 이번, 가을, 뉴욕, 여행, 가면,..."
5,2019.10.08.,노컷뉴스언론사 선정,"靑 김상조 ""젊은이들, 도전 기회조차 없는 사회에 분노""",일 수출규제로 직접적 기업 피해는 없지만 간접 피해 불확실성 존재경쟁력 제고 공급망...,일 수출규제로 직접적 기업 피해는 없지만 간접 피해 불확실성 존재경쟁력 제고 공급망...,"[일, 수출, 규제, 직접, 기업, 피해, 간접, 피해, 불확실, 존재, 경쟁력, ..."
6,2019.10.08.,뉴스1,터키에 케밥만 있는 줄 아니? 현지인 추천 미식여행지,터키판 성수동 카페거리부터 해산물 가득한 해안 도시까지카라코이 거리 전경 현지에서만...,터키판 성수동 카페거리부터 해산물 가득한 해안 도시까지카라코이 거리 전경 현지에서만...,"[터키, 판, 성수동, 카페, 거리, 해산물, 해안, 도시, 카라, 코, 거리, 전..."
7,2019.10.08.,한국일보,[자동차여행] 푸조 508의 연비를 체크하며 찾은 '무화과 축제',푸조 영암 무화과 축제를 찾았다 푸조 푸조의 새로운 시대를 여는 존재라 해도 과언이...,푸조 영암 무화과 축제를 찾았다 푸조 푸조의 새로운 시대를 여는 존재라 해도 과언이...,"[푸조, 영암, 무화과, 축제, 푸조, 푸조, 시대, 여, 존재, 해도, 과언, 것..."
8,2019.10.08.,뉴스1,"미식의 계절, 호텔가에선 해외 유명 셰프 초청 경쟁 중",영국 테오 랜달 셰프부터 미국 로니 엠보그까지인터컨티넨탈에서 초청하는 영국의 유명 ...,영국 테오 랜달 셰프부터 미국 로니 엠보그까지인터컨티넨탈에서 초청하는 영국의 유명 ...,"[영국, 테오, 랜달, 셰프, 미국, 로, 엠, 보그, 인터컨티넨탈, 초청, 영국,..."
9,2019.10.08.,연합뉴스,수출부진에 8월 상품수지 흑자폭 5년7개월 만에 최소,경상수지 흑자 달러서비스수지 개선에도 흑자폭 줄어내국인 해외증권투자 만에 첫 감소 ...,경상수지 흑자 달러서비스수지 개선에도 흑자폭 줄어내국인 해외증권투자 만에 첫 감소 ...,"[경상수지, 흑자, 달러, 서비스, 수지, 개선, 흑자, 폭, 국, 해외, 증권, ..."


In [13]:
df.to_csv(save_path, index = False)